In [1]:
import pandas as pd
import numpy as np
from itertools import product
import gc

In [2]:
df_items = pd.read_csv('./input/items.csv')
df_categories = pd.read_csv('./input/item_categories.csv')
df_shops = pd.read_csv('./input/shops.csv')
df_train = pd.read_csv('./input/sales_train_v2.csv')
df_test = pd.read_csv('./input/test.csv')

In [3]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype=='float64']
    int_cols = [c for c in df if df[c].dtype=='int64']

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int32)

    return df

In [4]:
df_train = df_train[df_train.item_price<100000]
df_train = df_train[df_train.item_cnt_day<=1000]

In [5]:
month_list = df_train['date_block_num'].unique()
index_cols = ['shop_id','item_id','date_block_num']

combination = []
for month in month_list:
    shop_list = df_train.loc[df_train['date_block_num']==month, 'shop_id'].unique()
    item_list = df_train.loc[df_train['date_block_num']==month, 'item_id'].unique()
    combination.append(list(product(*[shop_list, item_list,[month]])))

#Convert combinations to dataframe
combination = pd.DataFrame(np.vstack(combination),columns = index_cols,dtype=np.int32)

In [6]:
#Find item sold per month for each shop
gb = df_train.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(combination,gb,how='left',on=index_cols).fillna(0)

/home/chiayi/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [7]:
#shop-month aggregates
gb = df_train.groupby(['shop_id','date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum','target_shop_avg':'mean'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data,gb,how='left',on=['shop_id','date_block_num']).fillna(0)

/home/chiayi/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
#item-month aggregates
gb = df_train.groupby(['item_id','date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum','target_item_avg':'mean'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data,gb,how='left',on=['item_id','date_block_num']).fillna(0)

/home/chiayi/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [9]:
all_data = downcast_dtypes(all_data)
del combination, gb

#free memory
gc.collect()

168

In [10]:
tmp_test = df_test.copy()
tmp_test['date_block_num'] = 34
tmp_test.drop('ID',axis=1,inplace=True)

In [11]:
all_data = pd.concat([all_data,tmp_test],axis=0,ignore_index=True)

In [12]:
cols_to_rename = list(all_data.columns.difference(index_cols))

shift_range = [1,2,3,4,5,12]

for month_shift in shift_range:
    train_shift = all_data[cols_to_rename + index_cols].copy()
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    rename = lambda x:'{}_lag_{}'.format(x,month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns = rename)
    all_data = pd.merge(all_data, train_shift, on =index_cols,how='left').fillna(0)

del train_shift

In [17]:
all_data['sub_lag_1_2'] = all_data['target_lag_1'].subtract(all_data['target_lag_2'])
all_data['sub_item_lag_1_2'] = all_data['target_item_lag_1'].subtract(all_data['target_item_lag_2'])
all_data['sub_shop_lag_1_2'] = all_data['target_shop_lag_1'].subtract(all_data['target_shop_lag_2'])
all_data['add_sub_lag_1_2_3_4'] = (all_data['target_lag_1'].add(all_data['target_lag_2'])).subtract(all_data['target_lag_3'].subtract(all_data['target_lag_4']))
all_data['add_sub_item_lag_1_2_3_4'] = (all_data['target_item_lag_1'].add(all_data['target_item_lag_2'])).subtract(all_data['target_item_lag_3'] - all_data['target_item_lag_4'])
all_data['add_sub_shop_lag_1_2_3_4'] = (all_data['target_shop_lag_1'].add(all_data['target_shop_lag_2'])).subtract(all_data['target_shop_lag_3'] - all_data['target_shop_lag_4'])
all_data['mul_sub_lag_1_2'] = (all_data['target_lag_1']*2).subtract(all_data['target_lag_2'])
all_data['mul_sub_item_lag_1_2'] = (all_data['target_item_lag_1']*2).subtract(all_data['target_item_lag_2'])
all_data['mul_sub_shop_lag_1_2'] = (all_data['target_shop_lag_1']*2).subtract(all_data['target_shop_lag_2'])

In [20]:
# Category for each item
item_category_mapping = df_items[['item_id','item_category_id']].drop_duplicates()
all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')

In [21]:
#Mean Encoding for item_id and shop_id
for d in (all_data['date_block_num'].unique()):
    item_id_target_mean = all_data[all_data.date_block_num<d].groupby('item_id').target.mean()
    all_data.loc[all_data.date_block_num == d,'item_target_enc'] = all_data[all_data.date_block_num ==d]['item_id'].map(item_id_target_mean)
    
    shop_id_target_mean = all_data[all_data.date_block_num<d].groupby('shop_id').target.mean()
    all_data.loc[all_data.date_block_num == d,'shop_target_enc'] = all_data[all_data.date_block_num ==d]['shop_id'].map(shop_id_target_mean)
    
    category_id_target_mean = all_data[all_data.date_block_num<d].groupby('item_category_id').target.mean()
    all_data.loc[all_data.date_block_num == d,'category_target_enc'] = all_data[all_data.date_block_num ==d]['item_category_id'].map(category_id_target_mean)

In [22]:
# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num']>12]

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]]

# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num']

all_data = downcast_dtypes(all_data)
gc.collect();

In [23]:
all_data.to_csv('all_data_new.csv',index=False)